In [1]:
!pip install pandas

In [16]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [3]:
# !pip3 uninstall torch torchaudio -y

In [4]:
!git clone https://github.com/manujosephv/pytorch_tabular.git

fatal: destination path 'pytorch_tabular' already exists and is not an empty directory.


In [5]:
!pip install -U pytorch_tabular

In [6]:
!pip install -U pytorch_tabular[extra]

In [7]:
!cd pytorch_tabular && pip install .[extra]

Processing /home/jovyan/pytorch_tabular
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pytorch_tabular: filename=pytorch_tabular-1.1.0-py2.py3-none-any.whl size=160784 sha256=38b5874b380ae59560600d1133bd68c61458aaebf04957b41023cb647151a98b
  Stored in directory: /tmp/pip-ephem-wheel-cache-lah9cndp/wheels/2f/4d/da/dfc34a3fe737e14671481ec5c86ac494913ab96a2d6da19dcf
Successfully built pytorch_tabular
  Attempting uninstall: pytorch_tabular
    Found existing installation: pytorch_tabular 1.1.0
    Uninstalling pytorch_tabular-1.1.0:
      Successfully uninstalled pytorch_tabular-1.1.0


In [21]:
!conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia -y


done
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
\ WARNING conda.models.version:get_matcher(556): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.8.0.*, but conda is ignoring the .* and treating it as 1.8.0
WARNING conda.models.version:get_matcher(556): Using .* with relational operator is superfluous and deprecated and will be removed in a future version of conda. Your spec was 1.6.0.*, but conda is ignoring the .* and treating it as 1.6.0
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.3
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch-

In [8]:
import torch
import torch.nn as nn
import pandas as pd
import sklearn
import numpy
import torch.utils
from torch.utils.data import Dataset,DataLoader
import torch.utils.data
from tqdm.notebook import trange, tqdm
from statistics import mean, stdev, variance
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [9]:
class MLP_model (nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_fc=nn.Linear(input_dim,800)
        self.hidden_fc=nn.Linear(800,600)
        self.hidden_fc2=nn.Linear(600,400)
        self.hidden_fc3=nn.Linear(400,400)
        self.hidden_fc4=nn.Linear(400,400)
        self.hidden_fc5=nn.Linear(400,400)
        self.hidden_fc6=nn.Linear(400,400)
        self.hidden_fc7=nn.Linear(400,400)
        self.hidden_fc8=nn.Linear(400,400)
        self.hidden_fc9=nn.Linear(400,400)
        self.hidden_fc10=nn.Linear(400,100)

        self.output_fc=nn.Linear(100,output_dim)
        self.dropout = nn.Dropout(0.25)

    def forward(self,x):
       batch_size=x.shape[0]
       x = x.view(batch_size, -1)
       x=self.dropout(x)
       m_1=nn.functional.relu(self.input_fc(x))
       m_2=nn.functional.relu(self.hidden_fc(m_1))
       m_3=nn.functional.relu(self.hidden_fc2(m_2))
       m_4=nn.functional.relu(self.hidden_fc3(m_3))
       m_4=nn.functional.relu(self.hidden_fc4(m_4))
       m_4=nn.functional.relu(self.hidden_fc5(m_4))
       m_4=nn.functional.relu(self.hidden_fc6(m_4))
       m_4=nn.functional.relu(self.hidden_fc7(m_4))
       m_4=nn.functional.relu(self.hidden_fc8(m_4))
       m_4=nn.functional.relu(self.hidden_fc9(m_4))
       m_4=nn.functional.relu(self.hidden_fc10(m_4))
       m_4=self.dropout(m_4)

       #output=nn.functional.sigmoid(self.output_fc(m_4))
       output=(self.output_fc(m_4))

       return output 
    
""" 
class MLP_model (nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.input_fc=nn.Linear(input_dim,50)
        self.input_fc2=nn.Linear(50,25)
        self.output_fc=nn.Linear(25,output_dim)

    def forward(self,x):
       batch_size=x.shape[0]
       x = x.view(batch_size, -1)
       m_1=nn.functional.relu(self.input_fc(x))
       m_2=nn.functional.relu(self.input_fc2(m_1))

       output=self.output_fc(m_2)
       return output """
    


' \nclass MLP_model (nn.Module):\n    def __init__(self, input_dim, output_dim):\n        super().__init__()\n        self.input_fc=nn.Linear(input_dim,50)\n        self.input_fc2=nn.Linear(50,25)\n        self.output_fc=nn.Linear(25,output_dim)\n\n    def forward(self,x):\n       batch_size=x.shape[0]\n       x = x.view(batch_size, -1)\n       m_1=nn.functional.relu(self.input_fc(x))\n       m_2=nn.functional.relu(self.input_fc2(m_1))\n\n       output=self.output_fc(m_2)\n       return output '

In [10]:
def calculate_accuracy(y_pred, y):
  
    top_pred = y_pred.argmax(1)
    top_y= y.argmax(1)
    correct = top_pred.eq(top_y).sum()
    #print(top_pred)
    acc = correct.float() / y.shape[0]
    #print(correct)
    return acc

def model_train(model,dataloader,optimizer,citerion,device):
    epoch_loss=0
    epoch_acc=0
    model.train()

    for i,data in enumerate(dataloader):
        feature,label=data
        feature=feature.to(device)
        label=label.to(device)

        optimizer.zero_grad()
        y_pred=model(feature)
        loss=citerion(y_pred,label)
        acc=calculate_accuracy(y_pred,label)
        loss.backward()
        optimizer.step()

        epoch_loss+=loss.item()
        epoch_acc+=acc.item()

    return epoch_loss/len(dataloader),epoch_acc/len(dataloader)


In [11]:
def model_evaluate(model,dataloader,criterion,device):
    epoch_loss=0
    epoch_acc=0

    model.eval()
    with torch.no_grad():
        for i,data in enumerate(dataloader):
            feature,label=data
            feature=feature.to(device)
            label=label.to(device)

            y_pred=model(feature)

            loss=criterion(y_pred,label)
            acc=calculate_accuracy(y_pred,label)

            epoch_loss+=loss.item()
            epoch_acc+=acc.item()
    return epoch_loss/len(dataloader),epoch_acc/len(dataloader)


In [12]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(pd.__version__)
print(sklearn.__version__)

True
NVIDIA A100-SXM4-80GB
2.2.2
1.3.2


In [13]:
class train_dataset(Dataset):
    def __init__(self,csv_file_feature,csv_meta,training_target):
        inputdata=pd.read_csv(csv_file_feature)
        # Standardize the original data
        input_mean, input_std, input_var = inputdata.mean(), inputdata.std(), inputdata.var()
        new_std = 1
        # print("MEAN: \n", input_mean)
        # print("StandardDeviation: \n", input_std)
        # print("Variance: \n", input_var)
        self.base_frame=(inputdata-input_mean)/(input_std/new_std)
        # self.base_frame=inputdata
        self.meta_frame=pd.read_csv(csv_meta,dtype={'sample_name':'int32',
                      'apoe4':'string',
                      'bmi':'float32',
                      'diagnosis':'string',
                      'amlyoid_positive':'float32',
                      'visit':'int32'})
        self.target_frame=self.meta_frame[training_target]
        self.base_frame.drop(columns=['sample_name'])

    def __len__(self):
        return len(self.target_frame)
    
    def __getitem__(self, index):
        label=0
        if self.target_frame[index]=='positive':
            label=[1,0]
        else:
            label=[0,1]
        
        feature=self.base_frame.iloc[index]
        feature_tensor=torch.tensor(feature.values).float()

        label_tensor=torch.tensor(label).float()

        return feature_tensor,label_tensor


In [24]:
train_data=train_dataset('train_data.csv','Meta.csv','apoe4')
train_size= int(0.70*len(train_data))
test_size=len(train_data)-train_size

training_data,testing_data=torch.utils.data.random_split(train_data,[train_size,test_size])

training_features = [data[0] for data in training_data]
training_labels = [data[1]for data in training_data]
testing_features = [data[0] for data in testing_data]
testing_labels = [data[1] for data in testing_data]
# for ele in training_data:
#     print(ele)
# mean = torch.mean(training_data)
# mean


In [25]:
from pytorch_tabular.models import GANDALFConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

data_config = DataConfig(
    target=[
        training_labels
    ],  # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=training_features,
     #categorical_cols=cat_col_names,
)
trainer_config = TrainerConfig(
    batch_size=1024,
    max_epochs=100,
)
optimizer_config = OptimizerConfig()
model_config = GANDALFConfig(
    task="classification",
    gflu_stages=6,
    gflu_feature_init_sparsity=0.3,
    gflu_dropout=0.0,
    learning_rate=1e-3,
)

In [26]:
from pytorch_tabular import TabularModel

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
    verbose=True
)

ValidationError: Cannot convert 'list' to string: '[tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 0.]), tensor([1., 0.]), tensor([0., 1.]), tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.]), tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 0.]), tensor([1., 0.]), tensor([1., 0.]), tensor([1., 0.]), tensor([0., 1.]), tensor([1., 0.]), tensor([1., 0.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 1.]), tensor([0., 

In [ ]:
batch_size = 8
lr = 0.0001
EPOCHS = 50
n_splits = 5  # Number of folds



model=MLP_model(883,2) 

optimizer = torch.optim.Adam(model.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

training_dataloader=DataLoader(training_data,batch_size=8,shuffle=True)
testing_dataloader=DataLoader(testing_data,batch_size=8,shuffle=True)

# training_features = [data[0] for data in training_data]
# training_labels = [data[1]for data in training_data]
# training_feature_dataloader = DataLoader(training_features,batch_size=8,shuffle=True)
# training_label_dataloader = DataLoader(training_labels,batch_size=8,shuffle=True)

kf = KFold(n_splits=3, shuffle=True, random_state=42)

for epoch in range(EPOCHS):


    train_loss,train_acc=model_train(model,training_dataloader,optimizer,criterion,device)
    # scores = cross_val_score(model, training_feature_dataloader, training_label_dataloader, cv=kf)
    test_loss,test_acc=model_evaluate(model,testing_dataloader,criterion,device)
    print(f'\t E {epoch} Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t E {epoch} Test. Loss: {test_loss:.3f} |  Test. Acc: {test_acc*100:.2f}%')








In [ ]:
# --normalize--: not really helpful with the original classifier
# Cross validation
# improvement options (low dep-high dim)
# transformer model (pytorch?)
# encoding methods

